In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as xgb
from itertools import combinations

# Load the data
file_path = '../combined_output.csv'
data = pd.read_csv(file_path)

# Define features and target
X = data[['duration_ns_1', 'uptime_ns_1']]
y = data['duration_ns_2']

# Define the models
model_params = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "KNN": KNeighborsRegressor(),
    "Gaussian Process": GaussianProcessRegressor(),
    "SVR": SVR(),
    "Ridge": Ridge(),
    "Extra Trees": ExtraTreesRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": xgb.XGBRegressor()
}

# Ablation Experiment
results = {}
benchmarks = data['benchmark'].unique().tolist()

# Iterate through combinations of benchmarks to exclude
for r in range(1, 3):  # Exclude 1 or 2 benchmarks
    for excluded_benchmarks in combinations(benchmarks, r):
        remaining_data = data[~data['benchmark'].isin(excluded_benchmarks)]
        X_remaining = remaining_data[['duration_ns_1', 'uptime_ns_1']]
        y_remaining = remaining_data['duration_ns_2']
        
        # Split the data into training and testing sets (80-20 split)
        X_train, X_test, y_train, y_test = train_test_split(X_remaining, y_remaining, test_size=0.2, random_state=42)
        
        scores = {}
        for model_name, model in model_params.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            scores[model_name] = {'MSE': mse, 'R2': r2}
        
        results[excluded_benchmarks] = scores

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame.from_dict({k: {model: v for model, v in scores.items()} for k, scores in results.items()}, orient='index')

# Save results to a CSV file
results_df.to_csv('ablation_results.csv')


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the results CSV file
results_file_path = 'ablation_results.csv'
results_df = pd.read_csv(results_file_path)

# Reshape the DataFrame for easier plotting
melted_df = results_df.melt(id_vars='index', var_name='metric_model', value_name='value')

# Separate metric and model into two different columns
melted_df[['metric', 'model']] = melted_df['metric_model'].str.extract(r'(MSE|R2)_(.+)')

# Get unique models for plotting
models = melted_df['model'].unique()

# Set up the figure with subplots
fig, axes = plt.subplots(len(models), 2, figsize=(12, 5 * len(models)))

# Loop over models and create subplots for each
for i, model in enumerate(models):
    # Filter data for the current model
    model_data = melted_df[melted_df['model'] == model]

    # Plot MSE
    sns.barplot(data=model_data[model_data['metric'] == 'MSE'], x='index', y='value', ax=axes[i, 0])
    axes[i, 0].set_title(f'{model} - MSE')
    axes[i, 0].set_xticklabels(axes[i, 0].get_xticklabels(), rotation=45)
    axes[i, 0].set_ylabel('MSE')

    # Plot R²
    sns.barplot(data=model_data[model_data['metric'] == 'R2'], x='index', y='value', ax=axes[i, 1])
    axes[i, 1].set_title(f'{model} - R²')
    axes[i, 1].set_xticklabels(axes[i, 1].get_xticklabels(), rotation=45)
    axes[i, 1].set_ylabel('R²')

# Adjust the layout
plt.tight_layout()
plt.show()


KeyError: 'index'